In [ ]:
# 这个的脚本提取 精度更高

import os
import re
import json
import csv
from collections import Counter

FIELDS = [
    "Material Name",
    # 一定要加 Polymer A/B/C/D/E 字段
    "Polymer A Name", "Polymer A Type", "Polymer A Role",
    "Polymer B Name", "Polymer B Type", "Polymer B Role",
    "Polymer C Name", "Polymer C Type", "Polymer C Role",
    "Polymer D Name", "Polymer D Type", "Polymer D Role",
    "Polymer E Name", "Polymer E Type", "Polymer E Role",
    "Polymer Structure Type",
    "Copolymerized/Blended/Crosslinked/Filled With",
    "Material A Name", "Material A Type", "Material A Role",
    "Material B Name", "Material B Type", "Material B Role",
    "Material C Name", "Material C Type", "Material C Role",
    "Material D Name", "Material D Type", "Material D Role",
    "Material E Name", "Material E Type", "Material E Role",
    "Component Ratio",
    "Swelling Ratio",
    "Glass Transition",
    "Melting Point"
]


FIELD_ALIASES = {
    "Polymers": [
        # 通用聚合物及共聚物
        "polymer", "copolymer", "blend", "biopolymer",
        # 常见商用或合成聚合物
        "PLA", "polylactic acid", "PCL", "polycaprolactone",
        "PET", "polyethylene terephthalate", "PMMA", "polymethyl methacrylate",
        "PU", "polyurethane", "PA", "polyamide", "nylon",
        "PVC", "polyvinyl chloride", "PVA", "polyvinyl alcohol",
        "PAN", "polyacrylonitrile", "PVP", "polyvinylpyrrolidone",
        "PDMS", "polydimethylsiloxane", "PC", "polycarbonate",
        "PBS", "polybutylene succinate", "PGA", "poly(γ-glutamic acid)",
        "PE", "polyethylene", "PP", "polypropylene", "polyester",
        # 生物基/天然聚合物
        "GelMA", "gelatin methacrylate", "gelatin", "collagen",
        "chitosan", "alginate", "sodium alginate", "cellulose",
        "nanocellulose", "pectin", "lignin", "starch", "hyaluronic acid",
        "silk fibroin", "polyethylene glycol", "polydopamine",
        "polyacrylamide"
    ],
    "Additives or Modifiers": [
        # 基础功能添加剂
        "additive", "modifier", "plasticizer", "compatibilizer", "filler", "blend",
        "hybrid", "nanocomposite", "composite",
        # 无机纳米材料
        "nanoparticle", "nanofiller", "nanoclay", "TiO2", "SiO2", "ZnO", "CaCO3",
        "clay", "montmorillonite", "halloysite", "bentonite",
        # 有机/高分子共混物
        "PBAT", "PEG", "PHA", "PBSA", "PPC", "EVA", "PLA-g-MA",
        # 生物基/天然材料
        "cellulose nanocrystal", "microcrystalline cellulose", "hemicellulose",
        "soy protein", "wheat bran", "rice husk",
        # 碳基材料
        "CNT", "carbon nanotube", "carbon black", "graphene", "graphene oxide",
        "reduced graphene oxide",
        # 纤维及增强材料
        "fiber", "natural fiber", "glass fiber", "bamboo fiber", "hemp fiber",
        "basalt fiber", "jute fiber", "kenaf fiber",
        # 增塑剂
        "glycerol", "triacetin", "citrate", "ATBC", "TEC", "tributyl citrate",
        "polyethylene glycol",
        # 共混及相容化
        "blending", "blended", "copolymerized", "reactive compatibilization",
        "immiscible", "miscible",
        # 其他添加剂
        "antioxidant", "nucleating agent", "chain extender", "crosslinker",
        "UV stabilizer", "thermal stabilizer", "fire retardant", "flame retardant"
    ],
    "swelling ratio": [
        "swelling ratio",
        "swelling degree",
        "swelling capacity",
        "swelling behavior",
        "swelling percentage",
        "degree of swelling",
        "water uptake",
        "water absorption",
        "absorbency",
        "hydration ratio",
        "absorption capacity"
        ],
    "Glass Transition": ["glass transition", "glass temperature", "Glass Transition Temperature", "Tg"],
    "Melting Point": ["melting point", "melting temperature", "Tm"]
}
def normalize_field_name(key: str) -> str:
    key = key.strip()
    if key in FIELDS:
        return key
    base = re.sub(r"[（(][^）)]*[）)]", "", key)
    base = re.sub(r"[^\w\s\-/]", "", base)
    low = base.strip().lower()

    # 处理 Tg 和 Tm 映射
    if low in ["glass transition", "glass transition temperature", "tg"]:
        return "Glass Transition"
    if low in ["melting point", "melting temperature", "tm"]:
        return "Melting Point"

    # fallback FIELDS 对应
    for std in FIELDS:
        if low == std.lower():
            return std
    return base



def flatten_material_additives(mat_obj):
    flattened = {}
    if isinstance(mat_obj, dict):
        for mat_key, attrs in mat_obj.items():
            if isinstance(attrs, dict):
                for subk, subv in attrs.items():
                    flattened[f"{mat_key} {subk}"] = subv
            else:
                flattened[mat_key] = attrs
    elif isinstance(mat_obj, list):
        for one in mat_obj:
            flattened.update(flatten_material_additives(one))
    return flattened


def extract_json_blocks(content: str):
    blocks = re.findall(r"```json\s*(.*?)\s*```", content, flags=re.DOTALL)
    if blocks:
        return [b.strip() for b in blocks]
    
    # fallback: 提取 🧾 提取结果：后的一段 json 数据（支持 dict 或 list）
    m = re.search(r"🧾\s*提取结果：\s*([\s\S]+)", content)
    if not m:
        return []
    raw = m.group(1).strip()

    if raw.upper().startswith("NONE") or "无响应" in raw:
        return []

    # 试图提取完整 JSON list 或 dict 块（支持 [ {...} ] 或 {...}）
    try:
        # 直接尝试整体解析
        parsed = json.loads(raw)
        if isinstance(parsed, (list, dict)):
            return [raw]
    except json.JSONDecodeError:
        pass

    # fallback：仍尝试匹配 list 开头、结尾
    start = raw.find("[")
    end = raw.rfind("]") + 1
    if 0 <= start < end:
        return [raw[start:end]]

    # fallback：尝试匹配 dict
    start = raw.find("{")
    end = raw.rfind("}") + 1
    if 0 <= start < end:
        return [raw[start:end]]

    return []



def summarize_field_coverage(rows):
    counter = Counter()
    total = len(rows)           # 例如：26756
    for row in rows:
        for field in FIELDS:
            if row.get(field, "") not in ["", None, "null"]:
                counter[field] += 1

    print("\n📊 字段覆盖率统计：")
    for field in FIELDS:
        count = counter.get(field, 0)
        # 方法 A：直接用 .1% 自动乘以 100 并添加 % 符号
        print(f"{field:<45} : {count}/{total} ({count/total:.1%})")

        # 方法 B：手动算百分比
        # pct = count/total*100
        # print(f"{field:<45} : {count}/{total} ({pct:.1f}%)")


def convert_all_output_folders(root_folder: str, output_csv_path: str):
    fieldnames = ["来源"] + FIELDS
    all_rows = []

    for dirpath, _, filenames in os.walk(root_folder):
        if "extracted_output" not in dirpath.lower():
            continue
        for fn in filenames:
            if not fn.lower().endswith(".txt"):
                continue
            file_path = os.path.join(dirpath, fn)
            try:
                content = open(file_path, "r", encoding="utf-8", errors="ignore").read()
            except Exception:
                continue

            blocks = extract_json_blocks(content)
            if not blocks:
                continue

            for idx, raw_json in enumerate(blocks, start=1):
                try:
                    data = json.loads(raw_json)
                except json.JSONDecodeError:
                    continue
                if data is None or not isinstance(data, (dict, list)):
                    continue
                if isinstance(data, dict):
                    data = [data]

                for j, item in enumerate(data):
                    if not isinstance(item, dict):
                        continue

                    row = {"来源": f"{fn}_段{idx}_Material{j+1}"}
                    cleaned = {}

                    # 展开 Base Polymer(s)
                    base_polymer_data = item.get("Base Polymer(s)")
                    if isinstance(base_polymer_data, dict):
                        for label, subdata in base_polymer_data.items():
                            if isinstance(subdata, dict):
                                name = subdata.get("Name", "")
                                ptype = subdata.get("Type", "")
                                role = subdata.get("Role", "")
                                cleaned[f"{label} Name"] = name
                                cleaned[f"{label} Type"] = ptype
                                cleaned[f"{label} Role"] = role

                    for k, v in item.items():
                        if k == "Base Polymer(s)":
                            continue

                        nk = normalize_field_name(k)

                        # ✅ 正确处理 Other Material 字段
                        if nk.startswith("Other Material") and isinstance(v, dict):
                            mats = flatten_material_additives(v)
                            for mat_field, mat_val in mats.items():
                                cleaned[mat_field] = mat_val
                            continue

                        # ✅ 仅处理 Swelling Ratio 吸水率字段（独立提取）
                        if nk == "Swelling Ratio" and isinstance(v, dict):
                            val = v.get("value")
                            unit = v.get("unit")
                            std_nk = normalize_field_name(k)
                            if val is not None:
                                if isinstance(val, list) and len(val) == 2:
                                    val_str = f"{val[0]}–{val[1]}"
                                else:
                                    val_str = str(val)
                                cleaned[std_nk] = f"{val_str} {unit}".strip() if unit else val_str
                            else:
                                cleaned[std_nk] = ""
                            continue

                        # ✅ Component Ratio 提取
                        if nk == "Component Ratio" and isinstance(v, dict):
                            ratio_type = v.get("Type", "")
                            components = v.get("Components", {})
                            if isinstance(components, dict):
                                comp_parts = []
                                for comp_name, comp_val in components.items():
                                    if isinstance(comp_val, list):
                                        val_str = ", ".join(map(str, comp_val))
                                    else:
                                        val_str = str(comp_val)
                                    comp_parts.append(f"{comp_name}: {val_str}")
                                comp_str = "; ".join(comp_parts)
                                if ratio_type:
                                    cleaned["Component Ratio"] = f"{comp_str} ({ratio_type})"
                                else:
                                    cleaned["Component Ratio"] = comp_str
                            else:
                                cleaned["Component Ratio"] = ""
                            continue



                        # ✅ Tg / Tm 提取
                        if nk in ["Glass Transition", "Melting Point", "Glass Transition Temperature", "Melting Temperature"] and isinstance(v, dict):
                            val = v.get("value")
                            unit = v.get("unit")
                            std_nk = normalize_field_name(k)
                            if val is not None:
                                if isinstance(val, list) and len(val) == 2:
                                    val_str = f"{val[0]}–{val[1]}"
                                else:
                                    val_str = str(val)
                                cleaned[std_nk] = f"{val_str} {unit}".strip() if unit else val_str
                            else:
                                cleaned[std_nk] = ""
                            continue

                        # ✅ 其他常规字段提取
                        if isinstance(v, list):
                            raw_str = ", ".join(str(x) for x in v)
                        elif isinstance(v, dict) and "value" in v and "unit" in v:
                            val = v.get("value")
                            unit = v.get("unit")
                            if val is not None:
                                if isinstance(val, list) and len(val) == 2:
                                    val_str = f"{val[0]}–{val[1]}"
                                else:
                                    val_str = str(val)
                                raw_str = f"{val_str} {unit}".strip() if unit else val_str
                            else:
                                raw_str = ""
                        else:
                            raw_str = "" if v is None else str(v)

                        if re.fullmatch(r"\[\s*(?:\.\.|…)+\s*\]", raw_str):
                            raw_str = ""

                        cleaned[nk] = raw_str

                    # 字段对齐填充
                    for col in FIELDS:
                        if col in cleaned:
                            row[col] = cleaned[col]
                        else:
                            found = False
                            for k2 in cleaned:
                                if normalize_field_name(k2) == col:
                                    row[col] = cleaned[k2]
                                    found = True
                                    break
                            if not found:
                                row[col] = ""

                    all_rows.append(row)

    # ✅ 写出 CSV 文件
    with open(output_csv_path, "w", newline="", encoding="utf-8-sig") as cf:
        writer = csv.DictWriter(cf, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(all_rows)

    print(f"\n✅ 提取完成，共 {len(all_rows)} 条记录，保存至：{output_csv_path}")
    summarize_field_coverage(all_rows)


if __name__ == "__main__":
    root = r"Your_split_pdfs_Path"
    out_csv = os.path.join(root, "Extraction_Result.csv")
    convert_all_output_folders(root, out_csv)
